# 02 - Feature Calculations with NumPy (Part 1)

This notebook is the first part of a two-part exercise on feature extraction of 3D point clouds. The computed features are then later the input for a machine learning classification task, in which the class of each measured point is determined. Typical classes for aerial point clouds are ground, building, vegetation, low vegetation, movable objects (like cars), wires, and a few others.

The **objective** of the notebook is an introduction on the practical work with universal function of NumPy, to gain experience in form of a guided code implementation. It is important not only to copy and paste the lines of code, but to try to understand each step, maybe to try alternatives, to make mistakes, explore what would happen if you change something, to explore the results, and to learn and understand by trial and error. Only through hands-on engagement can you truly understand how to work with big data effectively and efficiently.

The **goal** is to calculate for each point of the 3D point cloud the eigenvalues and eigenvectors from a number of k (=20) nearest neighbor points.

See the lecture slides if you need to follow up on the computation.

In the second part of the exerices, eigenvalue and geometry based features are calculated from the output of this notebook.

The data used in this part of the course is located in the shared folder at "~/coursematerial/GIS/ISPRS/Vaihingen3DTraining.las" on the Jupyter hub server. All students have read access to this folder and can work directly with the data in this folder. You should not have any need to copy it into anywhere else. On the contrary, by making a copy on the server, you will use up your limited storage.

3D point clouds can be viewed with CloudCompare, an open source software you can download at https://www.danielgm.net/cc/

# Load LAS file

The 3D point cloud is stored in a file of format LAS, which is an open file format for LiDAR point clouds.

A 3D point cloud is just a unordered collection of points, typically with x,y,z-coordinates. The first value of the shape of the array gives you the number of points (n), and the second the number of coordinates (d).

When outputted with **print()**, an extract of the NumPy array (3D point cloud) is given. Because the real-world coordinates are rather large, it is helpful to set the print options to output the coordinates with 2 decimals only. (We could translate the 3D point cloud, so that the center of the point cloud is in the origin of the coordinate axis to make the coordinate values smaller. But this is not necessary for our exercise. However, translating geo data into a local coordinate system before performing any computations often leads to fewer numerical problems, since we then work with smaller numbers.) 

# k nearest neighbors

In order to calculate features of the points of a 3D point cloud, one must first determine for each point its k nearest neighbor points. This can be done e.g. by means of a kd-tree, which is build from the 3D coordinates of the points themselves using the constructor of the **KDTree** class of the library sklearn. (Although we provide 3D points as input in the form of an array of shape (n,3), where n is the number of points, the class KDTree works with any number of dimensions. However, the coordinate dimensions of the kd-tree and the ones of the query points must match (be the same). We cannot build a kd-tree with 3D coordinates and then provide query points with 2D coordinates.)

With the method **query()** of the class KDTree, the indices and distances of the k nearest neighbor points can be retrieved for given query points. For this purpose, the coordinates of n query points are provided as an array of shape (n,d) as the first argument to the method, where d is the number of dimensions. The indices and distances for all query points are then returned as an array of shape (n,k).

It is also possible to query just one single query point with this method, but it is still necessary to provide the one query point to the method **query()** as an array of shape (1,d). In the following example, we therefore extract from the (n,d) array of points the single query point by performing a slicing operation (, which unfortunately does not look as elegant anymore).

Using the indices of the neighbor points of the single query point, the coordinates of these neighbor points can be retrieved by an indexing operation from the original array.

The 2D array of indices stores for each of the n points the k indices of the k nearest neighbors. The order of the points in indices is always according to their distances to the query point.

And the 2D array of distances stores the respective distances of the neighbor points to the query points.

The first index number in the returned indices is always the index of the point itself, and the first distance is always 0.0, since we query with the same points that we build the kd-tree with. Therefore, the query always finds the point itself as one of its k nearest neighbors and includes it as the first point in the result.

Using the indices array (n,k) on the n input points with d coordinates retrieves an array of shape (n,k,d), which now includes for each of the n points the d coordinates of the k neighbor points.

The neighbor coordinates of a specific point, e.g. for the point at index 7, can then be retrieved by indexing.

Note that this gives the same array as first indexing point 7 in the indices array and then using the indices on the point array.

It is important to understand how the two approaches differ and what happens in the process. In the first approach, we get the neighbor coordinates for all points from the array of indices, and then extract the ones for point 7. In the other approach, we first retrieve the indices of neighbor points only for point 7, and then used this subarray of the indices to get the coordinates of the neighbors.

In the following, however, we continue to work with the complete array of neighbor coordinates, so that we can perform the calculations for all points at once.

# Covariance matrix

In order to compute Eigenvalue based features for a point in the 3D point cloud, we need to first retrieve its 20 neighbor points, build from the coordinates of these 20 points the covariance matrix, and then determined the eigenvalues and eigenvectors from this matrix. NumPy offers the relevant functions **cov()** and **eig()**, the latter within the module **linalg**, for this purpose.

Following is the computation for the point at index 17 that we retrieved above.

By using the indices of neighbor points returned from the query method of the kd-tree above for point 17, and apply it to indexing the original array, we notice that we get an array of shape (1, 20, 3). However, the **cov()** function expects an array of shape (n,d). We therefore have to get rid of the first dimension, which can be accomplished with slicing, reshaping, or squeezing the array.

One must be careful with squeeze, as it removes all dimensions of size 1. However, it is also possible to define which dimension to squeeze by providing the axis argument. (Squeezing a dimension that is not 1 will results in an error.)

Now, the **cov()** function can be used to calculate the covariance matrix of the group of k point coordinates. Because the variables (coordinates) are in the columns instead of the rows, the argument **rowvar** is set to false.

And the function **eig()** calculates from the covariance matrix the eigenvalues (w) and eigenvectors (v).

For the point at index 17, we could now calculate the eigenvalue based features from the arrays w and v.

In order to calculate the eigenvalues and eigenvectors for all points, we could do the computation within a for-loop over all points.

But first we import some modules to take the time this loop will take.

**Note:** the next cell will take almost 2 minutes to complete.

This approach using a for-loop is not very inefficient, as it does not make use of the vectorized functions of NumPy.

Unfortunately, both functions **cov()** and **eig()** are not vectorized, so we can only move the query of the kd-tree out of the loop.

(Because of the way we now index the neighbor points, we do not need to remove the first dimension of size 1. You can verify this by taking a look at the shape of the index and the result of the index.)

The computation time should now be down to under one minute, although the exact same thing is computed.

# Covariance matrix (vectorized)

In the following, a vectorized computation of the covariance matrices for all points is done directly with NumPy functions only.

Using the indices of neighbor points on the point cloud, we retrieve an array of shape (n,k,d). For each group of neighbor points, we then need to calculate the mean value for x, y, and z with the function **mean()**. 

However, we have to specify to the function, how the mean is to be calculated from the 3D array by defining the correct axis:
- If we provide no axis argument to the function, then the mean of all values of the point cloud is computed. The result is a scalar value.
- If we provide axis=0, the mean is calculated for each x-, y-, and z- coordinate individually as we want it, but for all points at the same (second) index within the group. The result is an array that gives the mean x, y, and z of all first neighbor points, all second neighbor points, all third neighbor points, etc. But we want the mean of all (first, second, third, ...) points within the same group.
- If we provide axis=1, then the mean x-, y-, z-coordinate of all points within the second dimension (axis=1) are computed as we want it.
- If we provide axis=1, then the mean x-, y-, z-coordinate for each point is calculated.

Check the shape of all these variations.

The one we need is the mean of all points within the same group of neighbor points, which is axis 1.

Next, we subtract the mean x-,y-,z-values (mean coordinates) per group element-wise from all points within the same group of neighbor points. As we have seen, the shape of the mean array is (n,d), and the shape of the neighbor points is (n,k,d). Therefore, the two arrays cannot just be subtracted as they are. We therefore have to increase the number of dimensions of the array of mean values from (n,d) to (n,k,d), repeating the mean value in the second dimension k times.

We therefore first reshape the mean array to (n,1,d), and then repeat the second dimension (axis 1) k times to get an array of (n,k,d) with the function **repeat()**. (Remember that reshaping an array does not change the number of elements in the array.)

Now we can subtract the two arrays element-wise from each other.

NumPy has a very nice feature that saves us from always transforming the arrays into exactly the same shape. In fact, it is sufficient to transform the array mean from the shape (n,d) to the shape (n,1,d). Now the two arrays are almost identical. The number of dimensions matches, but the size of one dimension (axis 1) is 1 in one array and k in the other array. In such a case, NumPy broadcasts the array with the dimension of size 1 so that the size matches the other array. In doing so, it repeats the one element as many times as needed to match the size of the dimension of the other array. In our case, it will repeat the mean k times to get an array of the shape (n,k,d).

This process of adapting the shape of one array to fit the shape of the other array is called broadcasting and it follows certain rules. In this exercise, however, we will not cover broadcasting in more detail and just show what is needed so that NumPy is able to broadcast the array to the appropriate shape.

The above subtraction of the mean from the neighbor points can be shortened to the following.

And instead of using the **reshape()** function, we can use the slicing operator with **newaxis** to introduce the additional dimension. This has the advantage that we do not even need to explicitely provide the final shape, but can just use the short version of using a single colon (:) for slicing all elements of all the other dimensions.

Putting all the above together into a one-liner, we end up with a very compact expression to center the point coordinates of each group by their mean coordinates.

For the covariance matrix, we regard each group of (mean centered) point coordinates as a matrix and multiply it by its transpose. (Actually, it is the other way around, we multiply the transpose of the matrix by itself.) 

If the input arrays have more dimensions than 2, then the arrays are considered as stacks of matrices, and only the last two dimensions are used for the actual matrix multiplication. In our case, NumPy will regard our arrays of shape (n,k,d) as a stack of n matrices of shape (k,d).

When transposing the array, we have to be careful to not just transpose the whole array of shape (n,k,d) as we would end up with an array of shape (d,k,n). This would be considered a stack of d matrices of shape (k,n), which would not fit the original (stack of n) matrices of shape (k,d). When we transpose, we can provide the way the dimensions should be transposed, leaving the first dimension as it is and only exchanging the dimensions 1 and 2. Then we have (a stack of n) matrices of shape (d,k), which can be multiplied by (a stack of n) matrices of shape (k,d).

Multiplying the transpose of the array by itself, we end up with n stacks of matrices of shape (d,d), or with an array of shape (n,d,d).

What is left is to divide each matrix element by the k-1 to have the final covariance matrices. Again, the arrays are divided element-wise and the arrays must match in their shape. However, a scalar value (as provided in the following) can be easily (and without ambiguity) broadcasted by NumPy into an array of any shape.

# Eigenvalues and eigenvectors

From this array of covariance matrices, we can calculate the eigenvalues and eigenvectors in the same way as above with a for-loop.

Putting everything together into the following cell, let us see how the final code looks like and how its computation time looks in comparison.

# Check correctness

The computation time should be a little over 20 seconds now, which is much faster than what we started with, showing that vectorized computations with NumPy can be very efficient even in Python.

But let us compare the resulting arrays of eigenvalues and eigenvectors from the previous computatuion and check that they are actually all the same. For this, we first compare the arrays element-wise with the == operator. 

As the result is an array of the same size as the original arrays, it is difficult to see if all values are the same, meaning that the operator == returned true. We therefore need to aggregate all Boolean values into a single value. The function **all()** tests whether all array elements evaluate to true (and returns true or false otherwise).

As w1 and w2 have been calculated with the same functions of NumPy, this can be expected. However, if we perform this test on our final version where we compute the covariance matrix ourselves, then this test fails.

To check where those values differ, we can use the **where()** function, which returns the indices of the array elements, where the condition we provide is true. Actually, the comparison operator returns an array of Boolean values, and the function **where()** returns the indices of the array where the Boolean value is true. As we want to know where the values are not equal, we have to use the not equal (!=) operator.

As the arrays we compare are 2D arrays, we get two arrays of indices, where the first array gives the indices of the rows, and the second array the indices of the columns. And we see that the values at index 0,2 are not equal. So, let us inspect them, outputting them with 20 decimals.

We notice that the values at the last column are slightly different. So, instead of using the condition of equal, we calculate the absolute value (with the function **abs()**) of the different between the array elements and compare the results with a very small value. NumPy provides small epsilon values that we can use, so that we do not need to define a value ourselves. We use the float32 epsilon value as the 64-bis epsilon would be too strict. (We could also just use 0.000001 as this would probably be sufficient too.)

Now that is surprising that the differences are also not small. So, let us use **where()** again to find the values once more.

There seems to be one row that makes problems, so let us print them.

OK, so the problem is that the first two eigenvalues (and therefore also the eigenvectors) are exchanged.

When checking the documentation of the function **eig()**, it does not say that the eigenvalues are guaranteed to be sorted. Although a lot of rows actually are. Let us find out how many rows are sorted and how many are not. Again, we use **where()**, but since we have 3 values, we need to have 2 greater (>) comparisons and need to combine their Boolean results (true, false) with a logical and operation, so that the result is only true if both inputs are also true. This leads to the following expression.

There seems to be a lot of rows not beeing sorted. So, instead of using **where()**, we can use the function **sum()** to sum up all values in the array. (On arrays with Boolean values, **sum()** (among others) regards true as value 1, and false as value 0.)

It seems that **eig()** generally does not sort the returned eigenvalues. And also that small changes in the covariance matrix that serve as input to **eig()** can also lead to a different order. Therefore, we better sort the eigenvalues and the eigenvectors in some order.

# Sorting eigenvalues and eigenvectors

Although NumPy provides the function **sort()** to sort an array, we cannot use it. The reason is that we need to first sort the eigenvalues, and then also need to exchange the eigenvectors in the same way as the eigenvalues were sorted. We therefore need to know the indices of the eigenvalues that would lead to sorted eigenvalues, so that we can apply these indices also on the eigenvectors. Luckily, NumPy provides a function **argsort()** that returns just these indices and leaves the input array unsorted.

Let us take a look at a simple example with two small arrays that store random integer values between 0 and 9.

By default, argsort will sort the array within the last axis. So, we do not provide an axis to sort by.

Take a look at the two arrays (rw and sort_ind) and verify that the indices returned by argsort would indeed sort the eigenvalues in ascending order.

Using the function **take_along_axis()**, we provide the array to be sorted and the indices from **argsort()**, in order to retrieve a sorted array along the provided axis. As we want to sort the eigenvalues within each row (column-wise), we have to specify that the sorted axis should be -1. (We could also specify the axis to be 1, as the array has 2 dimensions, and axis 0 is for the rows, and axis 1 for the columns. In this particular case, -1 and 1 index the same axis. But using -1 is typically the better choice to tell NumPy to take the last axis. Even if we decide to later increase the number of dimensions of the array, -1 would still be specifying the last axis.)

Because we later want the eigenvalues sorted in descending order (instead of ascending order), we need the indices in reverse order, which we can accomplish with the **flip()** method. (We need to tell **flip()** the axis we want to flip by, which is -1.) Our indices are therefore gotten by applying **flip()** on the result of **argsort()**.

Now, the indices applied to **take_along_axis()** gives us the eigenvalues in descending order.

The same indices can now be used to also sort the eigenvectors according to the values of the eigenvalues. There is the slight problem, that the indices are not in the same shape as the array of eigenvectors.

We therefore need to introduce another dimension to the index array, but it is a little tricky to get this axis right. Each eigenvector is stored in the last dimension (axis 2 or axis -1 depending on how you count it). We therefore must not sort by the last axis, as this would exchange the x-,y-, and z-values of the vectors, and we want to keep our vectors the same. Therefore, it must be axis 1, as this would sort the vectors (and not the components of the vectors). And axis 0 is just for the points and we do not need to sort the points.

Therefore, we introduce a more dimension at the end of the array of indices to sort by, and then take the values as before with **take_along_axis()** from the last axis. Broadcasting will make sure that the indices are repeated in the last dimension, so that all elements of the same vector go into the same array.

Now applied to our real array of eigenvalues and eigenvectors, the code looks like the following.

We can check again if the condition holds that all eigenvectors are sorted in descending order.

Let us also sort the eigenvalues and eigenvectors from the first implementation.

And verify that the arrays of eigenvalues and eigenvectors are the same from the first implementation (w1, v1) and the vectorized implementation (w, v).

As we can see, the eigenvalues are all the same and now in sorted (descending) order.

(Note that the eigenvectors cannot be verified as easily.)

# Normalize eigenvectors

In general, the eigenvectors should be normalized, meaning their length is 1. We can verify this by calculating the 2-norm with the function **norm()**.

Checking that all values are 1.0 does not work, because of numerical inaccuracies, as the following shows.

We would need to allow the length (norm) of the vectors to deviate a little (some small epsilon) from 1.

We can divide all the eigenvectors by the norm of the vectors to normalize the vectors (just to make sure). (Again, we need to add another axis to the norm.) And we call the resulting array the array of eigenvectors to continue with.

But as you can see from the following, this would not help to have exact vector lengths of 1.

To be able to reliably calculate features from eigenvalues, we have to make sure that eigenvalues do not have a value close to 0, because some features take the fraction of eigenvalues. And if an eigenvalue is in the denominator, it must not be close to zero as dividing by zero raises an error (exception).

# Summary

Bringing all the above together, we end up with the following code that:

- queries for each point the k nearest neighbor points
- builds the covariance matrix of neighbor points
- calculates the eigenvalues and eigenvectors
- sorts eigenvalues and eigenvectors
- makes sure that eigenvalues are greater than 0.0
- normalize the eigenvectors (not really necessary here)

In [ ]:
start_time = time.process_time()

# building the kd tree from the points of the point cloud
kd_tree_3d = KDTree(xyz)

# query for each point the k nearest neighbor points
distances3d, indices3d = kd_tree_3d.query(xyz, k=20)

# build the covariance matrix of neighbor points
xyz_minus_mean = xyz[indices3d] - np.mean(xyz[indices3d], axis=1)[:, np.newaxis, :]
xyz_xyz = np.matmul(np.transpose(xyz_minus_mean, axes=(0, 2, 1)), xyz_minus_mean)
xyz_cov = xyz_xyz / 19.0

# construct arrays to store eigenvalues and eigenvectors in
w = np.zeros((np.shape(xyz)[0], 3))
v = np.zeros((np.shape(xyz)[0], 3, 3))

for i in range(xyz.shape[0]):
    w[i], v[i,:] = np.linalg.eig(xyz_cov[i])
    
# sort eigenvalues and eigenvectors
idx = np.flip(np.argsort(w), axis=1)
w = np.take_along_axis(w, idx, axis=1)
v = np.take_along_axis(v, idx[:, :, np.newaxis], axis=1)    
    
# make sure eigenvalues are greater than 0.0    
eigenvalues = w
eigenvalues[eigenvalues <= np.finfo(np.float32).eps] = np.finfo(np.float32).eps

# normalize the eigenvectors
eigenvectors = v / np.linalg.norm(v, axis=-1)[:,:,np.newaxis]    
    
end_time = time.process_time()

print(timedelta(seconds=end_time - start_time))   

All calculations take a little more than 20 seconds, with most of the time spent determining the neighboring points and calculating the eigenvalues and eigenvectors.

The eigenvalues and eigenvectors can now be used to calculate the eigenvalue based features for each point. This is the task of the next exercise notebook.

If anything remains unclear to you, then it might help to
- check the documentation of the NumPy functions used
- think about why a specific axis is chosen and what effect it has on the computation
- follow up on the broadcasting
- make a short example with small random integer values that you can also calculate by hand

When doing vectorized computations with NumPy, it often helps to keep in mind what the dimensions stand for, which dimension (axis) to perform the computation on, and check the shape of the input and output arrays. Many misunderstandings and errors can be uncovered just by looking at the shape of the arrays.